In [ ]:
import os
from dotenv import load_dotenv
import sys
sys.path.append(r"C:\Users\Admin\Documents\GitHub\thesis\backend\backend")
from supabase_client import supabase
import pandas as pd

load_dotenv(r"C:\Users\Admin\Documents\GitHub\thesis\backend\.env")

# Get the Supabase credentials from environment variables
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

# Initialize the Supabase client
from supabase import create_client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

# Test query (optional, to confirm it's working)

data=[{'id': 1049, 'name': 'Ibuprocillin', 'category': 'Antiviral', 'dosage_form': 'Injection', 'strength': '337 mg', 'manufacturer': 'CSL Limited', 'indication': 'Infection', 'classification': 'Over-the-Counter', 'expiration_date': '2024-09-15', 'stocks': 185}] count=None


In [19]:
medicines = supabase.table('medicine_medicine').select().execute()
prescriptions = supabase.table('patient_prescription').select().execute()

med_df = pd.DataFrame(medicines.data)
pres_df = pd.DataFrame(prescriptions.data)
#print(pres_df['medication_id'].unique())

pres_df['start_date'] = pd.to_datetime(pres_df['start_date']) 
pres_df['month'] = pres_df['start_date'].dt.to_period('M').dt.to_timestamp()
pres_df = pres_df.merge(med_df, left_on='medication_id', right_on='id', how='left')
        

In [20]:
print(pres_df)

    id_x  medication_id  dosage      frequency start_date    end_date  \
0    143           1052  802 mg  Every 4 hours 2025-03-22  2025-03-28   
1    144           1050  333 mg  Every 8 hours 2025-03-21  2025-03-24   
2    145           1056  954 mg    Twice a day 2025-04-19  2025-04-30   
3    146           1050  333 mg    Twice a day 2025-04-19  2025-04-30   
4    150           1056  954 mg    Twice a day 2025-04-29  2025-04-29   
5    151           1056  954 mg    Twice daily 2025-04-29  2025-04-29   
6    152           1048  938 mg    Twice daily 2025-04-29  2025-04-29   
7    153           1056  954 mg    Twice Daily 2025-04-30  2025-04-30   
8    154           1064  861 mg    Twice Daily 2025-04-30  2025-04-30   
9    155           1056  954 mg    Twice daily 2025-05-04  2025-06-04   
10   156           1056  954 mg          twice 2025-05-16  2025-06-16   
11   157           1061  440 mg          Twice 2025-05-29  2025-06-29   
12   158           1056  954 mg            twe 2025

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

n_splits = 5
ts = TimeSeriesSplit(n_splits=n_splits)

for train_index, test_index in ts.split(pres_df):
    train_data, test_data = pres_df.iloc[train_index], pres_df.iloc[test_index]
    